In [1]:
import os

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import tableone
import warnings

In [2]:
pd.options.display.max_columns = 200

In [5]:
data_path = pd.read_csv(
    "/Users/mxposed/Documents/data_paths/ssc.csv",
    index_col=0
).value["clinical_data"]

# Create Supplementary table with stats over all 5 lobes

## 1. Load all processed data

In [6]:
hrct_scores = pd.read_csv(os.path.join(data_path, 'hrct_scoring_all.csv'), index_col=0)

Verify each patient has all 5 lobes

In [10]:
hrct_scores.groupby('study_code').size().eq(5).all()

True

In [20]:
hrct_scores.lobe = pd.Categorical(hrct_scores.lobe, categories=['LUL', 'RUL', 'RML', 'LLL', 'RLL'])

## X. Create table 1

In [ ]:
columns = [
    'lobe', 'ggo_score', 'fib_score'
]
nonnormal = [
    'ggo_score', 'fib_score'
]
renames = {
    'ggo_score': 'Ground-glass opacity score',
    'fib_score': 'Fibrosis score'
}
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    table1 = tableone.TableOne(
        hrct_scores.reset_index(),
        columns=columns,
        nonnormal=nonnormal,
        groupby=['lobe'],
        missing=False,
        overall=False,
        decimals={
            'ggo_score': 1,
            'fib_score': 1,
        },
        order={
            'lobe': ['LUL', 'RUL', 'RML', 'LLL', 'RLL']
        },
        # limit=1,
        rename=renames
    )

In [52]:
table1.tableone = table1.tableone.T

In [53]:
table1.tableone.index = table1.tableone.index.droplevel(0)

In [54]:
table1.tableone.index.name = 'level_0'

In [55]:
print(
    table1.tabulate(tablefmt="latex", headers=['Lobe'] + table1.tableone.columns.get_level_values(0).tolist())
        .replace('\\textbackslash{}', '\\')
        .replace('\\{', '{')
        .replace('\\}', '}')
)

\begin{tabular}{lrll}
\hline
 Lobe   &   n & Ground-glass opacity score, median [Q1,Q3]   & Fibrosis score, median [Q1,Q3]   \\
\hline
 LUL    &   9 & 1.0 [1.0,2.0]                                & 2.0 [0.0,2.0]                    \\
 RUL    &   9 & 1.0 [0.0,1.0]                                & 1.0 [0.0,2.0]                    \\
 RML    &   9 & 1.0 [0.0,2.0]                                & 1.0 [0.0,2.0]                    \\
 LLL    &   9 & 2.0 [2.0,3.0]                                & 3.0 [2.0,3.0]                    \\
 RLL    &   9 & 3.0 [2.0,3.0]                                & 3.0 [3.0,3.0]                    \\
\hline
\end{tabular}


Copying the above to [Overleaf project](https://www.overleaf.com/project/676e45db0ebdb3ce76c559fc) to get nice pdf table